# Quickstart

This notebook is just to make sure that everything works. Firstly, create a file called `.env` in this folder, and add the following content, obviously with your values. There are two ways to authenticate: using a token or using the Azure CLI (AAD authentication). The Azure CLI is the recommended way.

```
OPENAI_API_KEY=xxxxxx
OPENAI_API_BASE=https://xxxxxxx.openai.azure.com/
```

When authenticating using Azure CLI + Azure AD, open a terminal and run the following command: `az login --use-device-code`. This will open a browser window where you can authenticate. After authentication, use Option 1 code to run and you are good to go.

*(You could as well run the following command to get the token: `az account get-access-token --resource https://management.core.windows.net/ --query accessToken -o tsv`. Copy the token and add it to the `.env` file and then use the token to authenticate.)*

> **🔔 Please note that in all following notebooks you will need to set the authentication method accordingly!**

In [1]:
import os
import tiktoken
import openai
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from openai.embeddings_utils import cosine_similarity
from tenacity import retry, wait_random_exponential, stop_after_attempt

# Load environment variables
load_dotenv()

# Option 1 - Use Azure AD authentication with az cli (use az login in terminal)
default_credential = DefaultAzureCredential()
token = default_credential.get_token("https://cognitiveservices.azure.com/.default")
openai.api_type = "azure_ad"
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_key = token.token
openai.api_version = "2022-12-01"

# Option 2 - Using Access Key
# openai.api_type = "azure"
# openai.api_base = os.environ.get("OPENAI_API_BASE")
# openai.api_key = os.environ.get("OPENAI_API_KEY")
# openai.api_version = "2022-12-01"

# Define embedding model and encoding
EMBEDDING_MODEL = 'text-embedding-ada-002'
COMPLETION_MODEL = 'text-davinci-003'
encoding = tiktoken.get_encoding('cl100k_base')

Test if completitions work:

In [2]:
response = openai.Completion.create(engine="text-davinci-003",
                                    prompt="Knock knock.",
                                    temperature=0)
print(response.choices[0].text)



Who's there?

Boo.

Boo who?



Do it in a streaming fashion:

In [4]:
import sys
for resp in openai.Completion.create(engine='text-davinci-003', prompt='Give me 10 taglines for an ice cream shop', max_tokens=512, stream=True):
    sys.stdout.write(resp.choices[0].text)
    sys.stdout.flush()



1. Cool off & indulge: Our Ice Cream experience
2. Get your Scoop on!
3. Welcome to the Coolest Place in Town!
4. Deliciously creamy...Just What You Need!
5. The Perfect Sweet Treat!
6. A Cold Taste of Heaven!
7. A Las Vegas of Deliciousness!
8. Get Cone Crazy!
9. Refreshingly Irresistible!
10. Make Every Day a Sundae!

Test if embeddings work:

In [3]:
e = openai.Embedding.create(input="Hello World!", engine=EMBEDDING_MODEL)["data"][0]["embedding"]
print(e)

[-0.021054117009043694, -0.04000825434923172, 0.011857650242745876, 0.008105744607746601, -0.014736072160303593, -0.017451563850045204, 0.02107222005724907, 0.04529441148042679, -0.00010600320092635229, 0.006847566459327936, -0.03801689296960831, -0.006589594762772322, 0.018257159739732742, -0.013432635925710201, -0.00278111663646996, 0.011224035173654556, -0.0363151840865612, 0.012174457311630249, -0.015007620677351952, 0.006019341293722391, 0.015025724656879902, -0.0016949197743088007, -0.01721622236073017, 0.015840372070670128, 0.013939527794718742, -0.007101012393832207, -0.02217652089893818, 0.009250776842236519, -0.0031069759279489517, 0.00133738003205508, 0.026376482099294662, -0.016881311312317848, 0.008028805255889893, 0.02822301723062992, 0.0013690607156604528, 0.002591032302007079, -0.019660165533423424, 0.01131455134600401, 0.0004927486879751086, -0.011486533097922802, 0.02630406990647316, -0.014989517629146576, -0.00368628092110157, 0.011115415021777153, -0.021923074498772

Test if tokenizer works:

In [4]:
tokens = encoding.encode("Hello world!")
print(tokens)
print(len(tokens))

[9906, 1917, 0]
3


Also let's check if LangChain works:

In [13]:
import os
import openai
from dotenv import load_dotenv
from langchain.llms import AzureOpenAI
from langchain.embeddings import OpenAIEmbeddings

# Load environment variables
load_dotenv()

# Configure Azure OpenAI Service API
openai.api_type = "azure"
openai.api_version = "2022-12-01"
openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv("OPENAI_API_KEY")

# Create a completion
llm = AzureOpenAI(deployment_name="text-davinci-003")
joke = llm("Tell me a dad joke")
print(joke)

# Create embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1)
text = "This is a test document."

# Embed a single document
query_result = embeddings.embed_query(text)
print(query_result)

# Embed multiple documents at once
doc_result = embeddings.embed_documents([text, text])



Q: What did the baby corn say to the mama corn?
A: Where's my popcorn?
[-0.0031265635043382645, 0.01109372079372406, -0.004041045904159546, -0.011746684089303017, -0.0009627883555367589, 0.010767238214612007, -0.01039411686360836, -0.005273681599646807, -0.009874410927295685, -0.02617185190320015, 0.020335156470537186, 0.02254723757505417, -0.007509083021432161, 0.017283549532294273, -0.005976616870611906, 0.019175812602043152, 0.021267961710691452, -0.01563115231692791, 0.007655666675418615, -0.01841624267399311, -0.0006825303426012397, -0.006453013978898525, -0.010960462503135204, 0.017963165417313576, -0.022160788998007774, -0.003089917590841651, 0.014405179768800735, -0.029983028769493103, 0.01848287135362625, -0.007962159812450409, 0.010060971602797508, -0.019415676593780518, -0.003657929366454482, -0.024266263470053673, -0.005530203226953745, 0.0038411591667681932, -0.00570676987990737, -0.029476648196578026, 0.018576152622699738, -0.016630586236715317, 0.0075157457031309605, 0